# Aggregate demand data spatially and temporally

In [219]:
# imports
import pandas as pd
import geopandas as gpd
import numpy as np

import h3

In [220]:
# variables
VOI_INPUT_FILE_PATH = "../processed_data/voi_demand.pickle"
VOI_OUTPUT_FILE_PATH = "../processed_data/voi_demand_h3.pickle"
VOI_HOURLY_PICKUP_OUTPUT_FILE_PATH = "../processed_data/voi_pickup_demand_h3_hourly.pickle"
VOI_HOURLY_DROPOFF_OUTPUT_FILE_PATH = "../processed_data/voi_dropoff_demand_h3_hourly.pickle"


BOLT_INPUT_FILE_PATH = "../processed_data/bolt_demand.pickle"
BOLT_OUTPUT_FILE_PATH = "../processed_data/bolt_demand_h3.pickle"
BOLT_HOURLY_PICKUP_OUTPUT_FILE_PATH = "../processed_data/bolt_pickup_demand_h3_hourly.pickle"
BOLT_HOURLY_DROPOFF_OUTPUT_FILE_PATH = "../processed_data/bolt_dropoff_demand_h3_hourly.pickle"

GRID_CELLS_LIST_OUTPUT_FILE_PATH = "../processed_data/grid_cells_list.pickle"
GRID_COMMUNITY_MAP_OUTPUT_FILE_PATH = "../processed_data/grid_community_map.pickle"

# resolution 7: area ~ 5 161 293 m2 radius ~ 1406 m
# reosultion 8: area ~ 737 327 m2 radius ~ 530 m
# resolution 9: area ~ 105 332 m2 radius ~ 200 m
H3_RESOLUTION_GRID = 8
H3_RESOLUTION_COMMUNTIY = 6
STUTTGART_BOUNDS = [9.045, 48.7, 9.28, 48.86]
COMMUNITIES_TO_CONSIDER = ['861faa7afffffff', '861faa717ffffff', '861faa71fffffff', '861faa787ffffff', '861faa637ffffff', '861faa44fffffff', '861faa707ffffff', '861faa78fffffff', '861faa7a7ffffff']


In [221]:
voi_df = pd.read_pickle(VOI_INPUT_FILE_PATH)
bolt_df = pd.read_pickle(BOLT_INPUT_FILE_PATH)

In [222]:
def filter_df(df, bounds):
    """
    Filter the dataframe to only include rows within the specified bounds.
    """
    return df[(df['lon'] >= bounds[0]) & (df['lon'] <= bounds[2]) & 
              (df['lat'] >= bounds[1]) & (df['lat'] <= bounds[3])]

In [223]:
voi_df = filter_df(voi_df, STUTTGART_BOUNDS)
bolt_df = filter_df(bolt_df, STUTTGART_BOUNDS)

In [224]:
def add_h3_indicies(df):
    df["geometry"] = gpd.points_from_xy(df["lon"], df["lat"], crs="EPSG:4326")
    df["grid_index"] = df.apply(
        lambda x: h3.latlng_to_cell(x["lat"], x["lon"], H3_RESOLUTION_GRID), axis=1
    )
    df["community_index"] = df.apply(
        lambda x: h3.cell_to_parent(x["grid_index"], H3_RESOLUTION_COMMUNTIY), axis=1
    )
    # timestamp as mean of start range and end range
    df["timestamp"] = pd.to_datetime(
        (
            df["event_time_start_range"].view(np.int64)
            + df["event_time_end_range"].view(np.int64)
        )
        // 2
    )
    df.drop(columns=["event_time_start_range", "event_time_end_range"], inplace=True)
    df.drop(columns=["lon", "lat"], inplace=True)
    df.set_index("timestamp", inplace=True)
    return df

In [225]:
voi_df = add_h3_indicies(voi_df)
bolt_df = add_h3_indicies(bolt_df)

/tmp/ipykernel_2177050/1191234004.py:12: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df["event_time_start_range"].view(np.int64)
/tmp/ipykernel_2177050/1191234004.py:13: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  + df["event_time_end_range"].view(np.int64)
/tmp/ipykernel_2177050/1191234004.py:12: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df["event_time_start_range"].view(np.int64)
/tmp/ipykernel_2177050/1191234004.py:13: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  + df["event_time_end_range"].view(np.int64)


In [226]:
voi_df.head()

,bike_id,current_range_meters,current_fuel_percent,event_type,geometry,grid_index,community_index
timestamp,,,,,,,
2025-02-11 14:57:35,VOJ:Vehicle:ed086d78-6194-4a8b-a37c-390f1f07533c,69600.0,None,pickup,POINT (9.23589 48.81528),881faa7181fffff,861faa71fffffff
2025-02-11 14:57:35,VOJ:Vehicle:e1b69fc3-bf67-4eb8-a091-f2bc178d2f8d,76000.0,None,pickup,POINT (9.15024 48.81025),881faa713dfffff,861faa717ffffff
2025-02-11 14:57:35,VOJ:Vehicle:f1339d7c-0637-4a19-957d-fab5a843bda0,59200.0,None,pickup,POINT (9.16081 48.77264),881faa7a8dfffff,861faa7afffffff
2025-02-11 14:57:35,VOJ:Vehicle:149ddb57-a0b6-4bd2-a77a-aebb2132abfd,51200.0,None,pickup,POINT (9.15595 48.81764),881faa7107fffff,861faa717ffffff
2025-02-11 14:57:35,VOJ:Vehicle:2030e8bd-f73f-454a-9065-c129a5ec6ca9,76000.0,None,pickup,POINT (9.10643 48.72325),881faa782bfffff,861faa787ffffff


In [227]:
bolt_df.head()

,bike_id,current_range_meters,current_fuel_percent,event_type,geometry,grid_index,community_index
timestamp,,,,,,,
2025-02-11 14:42:07,BLT:Vehicle:740666c5-38af-4cd9-8ea8-d7a4b623294b,23800.0,0.44,pickup,POINT (9.23838 48.82612),881faa718bfffff,861faa71fffffff
2025-02-11 14:42:07,BLT:Vehicle:c736bf7f-9e1b-40df-af6c-afb04271040e,22400.0,0.42,pickup,POINT (9.07409 48.74804),881faa7b13fffff,861faa7b7ffffff
2025-02-11 14:42:07,BLT:Vehicle:f7b43738-0836-4443-9d7a-723976f84870,14000.0,0.30,pickup,POINT (9.18369 48.80967),881faa7133fffff,861faa717ffffff
2025-02-11 14:42:07,BLT:Vehicle:50f1520d-feea-47e2-afb3-56ec8064d9b5,46900.0,0.77,pickup,POINT (9.16575 48.74136),881faa78cdfffff,861faa78fffffff
2025-02-11 14:42:07,BLT:Vehicle:e1c6c470-1032-4ea6-94b6-e69de8492a97,35000.0,0.60,pickup,POINT (9.18678 48.78089),881faa7a8bfffff,861faa7afffffff


In [228]:
# filter the dataframes to only include the communities we are interested in
print(f"Number of rows in VOI DataFrame before filtering: {len(voi_df)}")
print(f"Number of rows in BOLT DataFrame before filtering: {len(bolt_df)}")


voi_df = voi_df[voi_df["community_index"].isin(COMMUNITIES_TO_CONSIDER)]
bolt_df = bolt_df[bolt_df["community_index"].isin(COMMUNITIES_TO_CONSIDER)]

print(f"Number of rows in VOI DataFrame after filtering: {len(voi_df)}")
print(f"Number of rows in BOLT DataFrame after filtering: {len(bolt_df)}")

print(f"Number of community indices in VOI DataFrame: {voi_df['community_index'].nunique()}")
print(f"Number of community indices in BOLT DataFrame: {bolt_df['community_index'].nunique()}")

print(f"Number of grid indices in VOI DataFrame: {voi_df['grid_index'].nunique()}")
print(f"Number of grid indices in BOLT DataFrame: {bolt_df['grid_index'].nunique()}")

Number of rows in VOI DataFrame before filtering: 533382
Number of rows in BOLT DataFrame before filtering: 377016
Number of rows in VOI DataFrame after filtering: 533274
Number of rows in BOLT DataFrame after filtering: 376810
Number of community indices in VOI DataFrame: 9
Number of community indices in BOLT DataFrame: 9
Number of grid indices in VOI DataFrame: 224
Number of grid indices in BOLT DataFrame: 229


In [229]:
# align the columns of the two dataframes
voi_columns = voi_df.grid_index.unique() 
bolt_columns = bolt_df.grid_index.unique()

intersection = list(set(voi_columns) & set(bolt_columns))
only_in_voj = list(set(voi_columns) - set(bolt_columns))
only_in_bolt = list(set(bolt_columns) - set(voi_columns))
all_columns = sorted(list(set(voi_columns) | set(bolt_columns)))
print("Total grid cells: ", len(all_columns))
print("Columns in both dataframes: ", len(intersection))
print("Columns only in VOI dataframe: ", len(only_in_voj))
print("Columns only in BOLT dataframe: ", len(only_in_bolt))

Total grid cells:  260
Columns in both dataframes:  193
Columns only in VOI dataframe:  31
Columns only in BOLT dataframe:  36


In [230]:
# save the dataframes to pickle files
voi_df.to_pickle(VOI_OUTPUT_FILE_PATH)
bolt_df.to_pickle(BOLT_OUTPUT_FILE_PATH)

In [231]:
grid_cells_df = pd.DataFrame(all_columns, columns=["grid_index"])
grid_cells_df["geometry"] = grid_cells_df.apply(
    lambda x: h3.cell_to_boundary(x["grid_index"]), axis=1
)
grid_cells_df.to_pickle(GRID_CELLS_LIST_OUTPUT_FILE_PATH)    
grid_cells_df.head()

,grid_index,geometry
0,881faa4485fffff,"((48.833396536059425, 9.050864230899897), (48...."
1,881faa4487fffff,"((48.83200932584397, 9.062893592658527), (48.8..."
2,881faa4493fffff,"((48.84129862635126, 9.10439595805226), (48.83..."
3,881faa4497fffff,"((48.83526513835251, 9.095673690358426), (48.8..."
4,881faa44b1fffff,"((48.82319564428952, 9.078234669656075), (48.8..."


In [232]:
grid_commmunity_map_df = pd.DataFrame(all_columns, columns=["grid_index"])
grid_commmunity_map_df["community_index"] = grid_commmunity_map_df.apply(
    lambda x: h3.cell_to_parent(x["grid_index"], H3_RESOLUTION_COMMUNTIY), axis=1
)
grid_commmunity_map_df = grid_commmunity_map_df[grid_commmunity_map_df["community_index"].isin(COMMUNITIES_TO_CONSIDER)]
grid_commmunity_map_df.to_pickle(GRID_COMMUNITY_MAP_OUTPUT_FILE_PATH)
grid_commmunity_map_df.head()

,grid_index,community_index
0,881faa4485fffff,861faa44fffffff
1,881faa4487fffff,861faa44fffffff
2,881faa4493fffff,861faa44fffffff
3,881faa4497fffff,861faa44fffffff
4,881faa44b1fffff,861faa44fffffff


In [233]:
grid_commmunity_map_df["community_index"].unique(), grid_commmunity_map_df["community_index"].nunique(), len(grid_commmunity_map_df)

(array(['861faa44fffffff', '861faa637ffffff', '861faa707ffffff',
        '861faa717ffffff', '861faa71fffffff', '861faa787ffffff',
        '861faa78fffffff', '861faa7a7ffffff', '861faa7afffffff'],
       dtype=object),
 9,
 260)

In [234]:
# group by grid index and sum the demand hourly
def group_by_hour(df):
    grouped_df = df.resample('h').grid_index.value_counts().unstack().fillna(0)
    # add mising columns
    for col in all_columns:
        if col not in grouped_df.columns:
            grouped_df[col] = 0
    
    # reorder columns
    grouped_df = grouped_df[all_columns]
    return grouped_df
    

In [235]:
voi_pickup_df = group_by_hour(voi_df[voi_df["event_type"] == "pickup"])
voi_dropoff_df = group_by_hour(voi_df[voi_df["event_type"] == "dropoff"])

bolt_pickup_df = group_by_hour(bolt_df[bolt_df["event_type"] == "pickup"])
bolt_dropoff_df = group_by_hour(bolt_df[bolt_df["event_type"] == "dropoff"])

In [236]:
voi_dropoff_df.head()

grid_index,881faa4485fffff,881faa4487fffff,881faa4493fffff,881faa4497fffff,881faa44b1fffff,881faa44b3fffff,881faa44b7fffff,881faa44b9fffff,881faa44bbfffff,881faa44bdfffff,...,881faa7ad9fffff,881faa7adbfffff,881faa7addfffff,881faa7ae1fffff,881faa7ae3fffff,881faa7ae5fffff,881faa7ae7fffff,881faa7ae9fffff,881faa7aebfffff,881faa7aedfffff
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-02-11 14:00:00,0.0,0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2025-02-11 15:00:00,0.0,0,0.0,0.0,27.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
2025-02-11 16:00:00,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0
2025-02-11 17:00:00,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0
2025-02-11 18:00:00,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0


In [237]:
bolt_dropoff_df.head()

grid_index,881faa4485fffff,881faa4487fffff,881faa4493fffff,881faa4497fffff,881faa44b1fffff,881faa44b3fffff,881faa44b7fffff,881faa44b9fffff,881faa44bbfffff,881faa44bdfffff,...,881faa7ad9fffff,881faa7adbfffff,881faa7addfffff,881faa7ae1fffff,881faa7ae3fffff,881faa7ae5fffff,881faa7ae7fffff,881faa7ae9fffff,881faa7aebfffff,881faa7aedfffff
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-02-11 14:00:00,0,0,0,0,0,0.0,0,0,0,0,...,0.0,0.0,0.0,1.0,1.0,0,0.0,0.0,1.0,0.0
2025-02-11 15:00:00,0,0,0,0,0,0.0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0,0.0,0.0,3.0,0.0
2025-02-11 16:00:00,0,0,0,0,0,0.0,0,0,0,0,...,1.0,2.0,3.0,2.0,3.0,0,0.0,0.0,2.0,0.0
2025-02-11 17:00:00,0,0,0,0,0,0.0,0,0,0,0,...,1.0,1.0,2.0,2.0,2.0,0,0.0,1.0,3.0,0.0
2025-02-11 18:00:00,0,0,0,0,0,0.0,0,0,0,0,...,1.0,0.0,1.0,1.0,3.0,0,0.0,1.0,2.0,0.0


In [238]:
len(voi_pickup_df.columns), len(bolt_pickup_df.columns), len(voi_dropoff_df.columns), len(bolt_dropoff_df.columns)

(260, 260, 260, 260)

In [239]:
# Save the dataframes to pickle files
voi_pickup_df.to_pickle(VOI_HOURLY_PICKUP_OUTPUT_FILE_PATH)
voi_dropoff_df.to_pickle(VOI_HOURLY_DROPOFF_OUTPUT_FILE_PATH)

bolt_pickup_df.to_pickle(BOLT_HOURLY_PICKUP_OUTPUT_FILE_PATH)
bolt_dropoff_df.to_pickle(BOLT_HOURLY_DROPOFF_OUTPUT_FILE_PATH)

In [240]:
bolt_pickup_df.index[0], bolt_pickup_df.index[-1]

(Timestamp('2025-02-11 14:00:00'), Timestamp('2025-04-28 09:00:00'))

In [241]:
len(voi_pickup_df), len(voi_dropoff_df), len(bolt_pickup_df), len(bolt_dropoff_df)

(1748, 1748, 1793, 1793)